# Week 7 Assignment
## Indian Air Quality Data

**Context:** Since industrialization, there has been an increasing concern about environmental pollution. As mentioned in the WHO report 7 million premature deaths annually are linked to air pollution, air pollution is the world's largest single environmental risk. Moreover as reported in the NY Times article, India’s Air Pollution Rivals China’s as World’s Deadliest it has been found that India's air pollution is deadlier than China's. We will explore India’s air pollution levels more granularly using this dataset.

**Content:** This data is combined(across the years and states) and is largely a clean version of the Historical Daily Ambient Air Quality Data released by the Ministry of Environment and Forests and Central Pollution Control Board of India under the National Data Sharing and Accessibility Policy (NDSAP).

In [1]:
# importing libraries
import numpy as npy
import pandas as pds
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning) # ignore warning messages
warnings.filterwarnings(action='ignore', category=UserWarning)

In [5]:
air = pds.read_csv('E:/PrepInsta Winter Internship Program/Week 7/data-1.csv')

C:\Users\debar\AppData\Local\Temp\ipykernel_9960\2827644914.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  air = pd.read_csv('E:/PrepInsta Winter Internship Program/Week 7/data-1.csv')


In [3]:
air.sample(3)

,stn_code,sampling_date,state,location,agency,type,so2,no2,rspm,spm,location_monitoring_station,pm2_5,date
402345,362,August,Uttar Pradesh,Varanasi,Uttar Pradesh State Pollution Control Board,"Residential, Rural and other Areas",20.0,31.0,155.0,NaN,"Regional Office, Jawahar Nagar, Varanasi.",NaN,41512.0
303299,NaN,January,Punjab,Ludhiana,NaN,Residential and others,10.7,40.0,252.0,NaN,PPCB Office Bldg.,NaN,39827.0
91779,NaN,January,Gujarat,Ahmedabad,NaN,Residential and others,12.7,17.1,83.0,180.0,Cadilla Bridge Narol,NaN,39844.0


In [6]:
# retrieving the unique values in the 'type' column of the "air" dataframe
air['type'].unique()

array(['Residential, Rural and other Areas', 'Industrial Area', nan,
       'Sensitive Area', 'Industrial Areas', 'Residential and others',
       'Sensitive Areas', 'Industrial', 'Residential', 'RIRUO',
       'Sensitive'], dtype=object)

In [8]:
# replacing multiple values in the 'type' column of the dataframe
air['type'].replace('Residential, Rural and other Areas','Residential',inplace = True)
air['type'].replace('Residential and others','Residential',inplace = True)
air['type'].replace('Industrial Areas','Industrial',inplace = True)
air['type'].replace('Industrial Area','Industrial',inplace = True)
air['type'].replace('Sensitive Area','Sensitive',inplace = True)
air['type'].replace('Sensitive Areas','Sensitive',inplace = True)

In [9]:
# retrieving the unique values in the 'state' column of the 'air' DataFrame
air['state'].unique()

array(['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar',
       'Chandigarh', 'Chhattisgarh', 'Dadra & Nagar Haveli',
       'Daman & Diu', 'Delhi', 'Goa', 'Gujarat', 'Haryana',
       'Himachal Pradesh', 'Jammu & Kashmir', 'Jharkhand', 'Karnataka',
       'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya',
       'Mizoram', 'Nagaland', 'Odisha', 'Puducherry', 'Punjab',
       'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Uttar Pradesh',
       'Uttarakhand', 'Uttaranchal', 'West Bengal',
       'andaman-and-nicobar-islands', 'Lakshadweep', 'Tripura'],
      dtype=object)

In [10]:
# Replacing a specific value in the 'state' column for maintaining consistency
air['state'].replace('andaman-and-nicobar-islands', 'Andaman and Nicobar Islands', inplace=True)

In [11]:
# Converting the 'date' column to datetime format and extract the 'year' column
air['date'] = pds.to_datetime(air['date'],format='mixed')
air['year'] = air['date'].dt.year

In [12]:
# Filling the missing 'year' values using ffill and converting them to integer type
air['year'].fillna(method='ffill', inplace=True)
air['year'] = air['year'].astype(int)

In [13]:
# checking null values in year column
print("Null Values in Year column:", air['year'].isnull().sum())

Null Values in Year column: 0


In [14]:
# Creating a separate dataFrame to show the proportion of missing values of each column
missing = pds.DataFrame(air.isna().sum() / len(air))
missing.columns = ['Proportion']

In [15]:
# Displaying the columns sorted by the proportion of missing values in descending order
print(missing.sort_values(by='Proportion', ascending=False).round(3))

                             Proportion
pm2_5                             0.979
spm                               0.545
agency                            0.343
stn_code                          0.331
rspm                              0.092
so2                               0.080
location_monitoring_station       0.063
no2                               0.037
type                              0.012
date                              0.000
sampling_date                     0.000
location                          0.000
state                             0.000
year                              0.000


#### Defining a function `get_state_median` that takes a state as an argument and calculates and prints the median values for "Industrial, Residential, and Sensitive" types for that state using the 'air' dataFrame. The function returns their median values.

In [16]:
pollutants = ['pm2_5', 'no2', 'so2', 'spm', 'rspm']

def get_state_median(state):
    # calculate the median values for no2 for the given state, grouped by type
    state_medians = air[air['state'] == state].groupby('type')[pollutants].median().reset_index()

    # return the state_medians dataframe
    return state_medians

In [17]:
get_state_median('Andhra Pradesh')

,type,pm2_5,no2,so2,spm,rspm
0,Industrial,NaN,22.2,5.4,214.0,76.0
1,Residential,NaN,20.0,5.0,192.0,78.0
2,Sensitive,NaN,13.0,4.6,130.0,51.0


In [40]:
median_values = get_state_median('Andhra Pradesh')

In [25]:
# Fill missing 'so2' values in 'Andhra Pradesh' for Industrial, Residential, and Sensitive types
air.loc[(air['state'] == 'Andhra Pradesh') & (air['type'].isin(['Industrial', 'Residential', 'Sensitive'])), 'so2'] = median_values['so2']

In [26]:
# Fill missing 'no2' values in 'Andhra Pradesh' for Industrial, Residential, and Sensitive types
air.loc[(air['state'] == 'Andhra Pradesh') & (air['type'].isin(['Industrial', 'Residential', 'Sensitive'])), 'no2'] = median_values['so2']

In [41]:
# Print the number of missing values in the 'rspm' and 'spm' columns
print("Missing Values in rspm:", air['rspm'].isnull().sum())
print("Missing values in spm:" ,air['spm'].isnull().sum())

Missing Values in rspm: 40222
Missing values in spm: 237387


#### Grouping the 'air' dataFrame by `location` and `type`, then using for loops to iterate through the groups. Sorting the values by 'date' and forward-fill missing values in the 'rspm' and 'spm' columns. The results are concatenated into a new dataFrame named data.

In [28]:
# Group 'air' DataFrame by 'location' and 'type' and create a new dataFrame data
df1 = dict(list(air.groupby(['location', 'type'])))
data = pds.DataFrame()

# Iterate through groups, sort by 'date', and forward-fill 'rspm' and 'spm' values
for key in df1:
    df2 = df1[key].sort_values('date')
    df2['rspm'].fillna(method='ffill', inplace=True)
    df2['spm'].fillna(method='ffill', inplace=True)
    data = pd.concat([data, df2])

#### Grouping the `data` dataFrame by **location** and **type**, then iterate through the groups. Within each group, we sort the values by 'date' and backward-fill missing values in the 'rspm' and 'spm' columns. The results are concatenated into a new DataFrame named **`data1`**.

In [29]:
# Group 'data' DataFrame by 'location' and 'type' and create a new DataFrame with backward-filled 'rspm' and 'spm' values
df1 = dict(list(data.groupby(['location', 'type'])))
data1 = pds.DataFrame()

# Iterate through groups, sort by 'date', and backward-fill 'rspm' and 'spm' values
for key in df1:
    df2 = df1[key].sort_values('date')
    df2['rspm'].fillna(method='bfill', inplace=True)
    df2['spm'].fillna(method='bfill', inplace=True)
    data1 = pd.concat([data1, df2])

In [43]:
# Print the number of missing values in the 'rspm' and 'spm' columns of the 'data1' DataFrame
print("Missing values in the 'data1' df")
print("rspm:", data1['rspm'].isnull().sum())
print("spm:", data1['spm'].isnull().sum())

Missing values in the 'data1' df
rspm: 4102
spm: 47909


#### Grouping the `data1` dataFrame by 'state' and 'type', then iterate through the groups. Within each group, missing values in 'rspm' and 'spm' columns will be filled with the group-wise medians. The results are concatenated into a new dataFrame named **`data2`**.

In [31]:
# Group 'data1' DataFrame by 'state' and 'type' and create a new DataFrame with median-filled 'rspm' and 'spm' values
df1 = dict(list(data1.groupby(['state', 'type'])))
data2 = pds.DataFrame()

# Iterate through groups and fill missing 'rspm' and 'spm' values with group-wise medians
for key in df1:
    df2 = df1[key]
    df2['rspm'].fillna(df2['rspm'].median(), inplace=True)
    df2['spm'].fillna(df2['spm'].median(), inplace=True)
    data2 = pd.concat([data2, df2])

In [44]:
# Print the number of missing values in the 'rspm' and 'spm' columns of the 'data2' DataFrame
print("Missing values in the 'data2' df")
print("rspm:", data2['rspm'].isnull().sum())
print("spm:", data2['spm'].isnull().sum())

Missing values in the 'data2' df
rspm: 182
spm: 1972


#### Grouping the `data2` dataFrame by 'type', then iterate through the groups. Within each group, missing values in 'rspm' and 'spm' columns are filled with the group-wise medians. The results are concatenated into a new dataFrame named **`data3`**.

In [33]:
# Group 'data2' DataFrame by 'type' and create a new DataFrame with median-filled 'rspm' and 'spm' values
df1 = dict(list(data2.groupby('type')))
data3 = pds.DataFrame()

# Iterate through groups and fill missing 'rspm' and 'spm' values with group-wise medians
for key in df1:
    df2 = df1[key]
    df2['rspm'].fillna(df2['rspm'].median(), inplace=True)
    df2['spm'].fillna(df2['spm'].median(), inplace=True)
    data3 = pds.concat([data3, df2])

In [45]:
# Print the number of missing values in the 'rspm' and 'spm' columns of the 'data3' DataFrame
print("Missing values in the data3 df")
print("rspm:", data3['rspm'].isnull().sum())
print("spm:", data3['spm'].isnull().sum())

Missing values in the data3 df
rspm: 0
spm: 1304


In [35]:
# Display the count of each type in the 'data3' DataFrame
data3['type'].value_counts()

type
Residential    265963
Industrial     148071
Sensitive       15011
RIRUO            1304
Name: count, dtype: int64

#### Resetting the index of the **`data3`** dataFrame and dropping unnecessary columns to obtain a cleaner dataset.

In [36]:
# Reset index and drop unnecessary columns from the 'data3' DataFrame
data3.reset_index(inplace=True)
data3.drop(columns=['index', 'stn_code', 'sampling_date', 'agency', 'location_monitoring_station'], inplace=True)
data3.head()

,state,location,type,so2,no2,rspm,spm,pm2_5,date,year
0,Andhra Pradesh,Hyderabad,Industrial,5.0,5.0,90.3,82.0,NaN,1990-01-02,1990
1,Andhra Pradesh,Hyderabad,Industrial,NaN,NaN,90.3,82.0,NaN,1990-01-03,1990
2,Andhra Pradesh,Hyderabad,Industrial,NaN,NaN,90.3,82.0,NaN,1990-01-04,1990
3,Andhra Pradesh,Hyderabad,Industrial,NaN,NaN,90.3,82.0,NaN,1990-01-05,1990
4,Andhra Pradesh,Hyderabad,Industrial,NaN,NaN,90.3,82.0,NaN,1990-01-06,1990


In [37]:
# Check for missing values in the 'data3' DataFrame
data3.isnull().sum()  

state            0
location         0
type             0
so2          59271
no2          41067
rspm             0
spm           1304
pm2_5       421035
date             4
year             0
dtype: int64

#### Using the `to_csv` method to save a copy of the final cleaned data stored in the 'data3' dataFrame to a CSV file named 'air_quality_cleaned.csv'.

In [46]:
data3.to_csv('air_quality_cleaned.csv', index=False)